In [1]:
import ee
import geemap.core as geemap
import folium

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-samagrabarnwal')

In [3]:
# Define your area of interest with the given coordinates
min_lat = 0
max_lat = 25
min_lon = 60
max_lon = 100
aoi = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

# Define the date range for the dataset
start_date = '2014-05-01'
end_date = '2014-05-14'

In [4]:
dataset = ee.ImageCollection('NOAA/CDR/SST_PATHFINDER/V53') \
                .filterDate(start_date, end_date) \
                .filterBounds(aoi)
sst = dataset.select('sea_surface_temperature')

In [5]:
# Compute the mean sea surface temperature over the specified time period
mean_sst = sst.mean()

In [21]:
# Set visualization parameters for sea surface temperature
vis_params = {
    'min': 0.0,
    'max': 2500.0,  # Adjust based on your knowledge of expected SST values
    'palette': [
        '030d81', '0519ff', '05e8ff', '11ff01', 'fbff01', 'ff9901', 'ff0000',
        'ad0000'
    ]
}

In [22]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

In [23]:
# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object centered on your area of interest
my_map = folium.Map(location=[(min_lat + max_lat) / 2, (min_lon + max_lon) / 2], zoom_start=5)

# Add the mean sea surface temperature layer to the map
my_map.add_ee_layer(mean_sst, vis_params, 'Mean Sea Surface Temperature')

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
my_map

In [25]:
# Calculate standard deviation of SST
sst_stddev = sst.reduce(ee.Reducer.stdDev())

# Add standard deviation layer to the map
my_map.add_ee_layer(sst_stddev, vis_params, 'SST Standard Deviation')

In [26]:
my_map